# CLAPES UC

## Extracción de links

Seminarios

In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de los divs con clase 'col-sm-6 mb-32'
        articles = driver.find_elements(By.CSS_SELECTOR, "div.col-sm-6.mb-32")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "span img")
                image_url = img.get_attribute("src")
                # Asegurarse de que la URL de la imagen sea completa
                if image_url and not image_url.startswith("http"):
                    image_url = "https://example.com" + image_url  # Reemplaza con el dominio adecuado
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h4.h3 a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "p.p-size--sm span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

In [49]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_to_bottom(driver):
    """Desplaza la página hasta el final y espera un segundo para cargar el contenido dinámico."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Esperar un segundo para que la página cargue

def extract_articles_from_url2(driver, url, existing_df=None):
    try:
        driver.get(url)

        # Desplazar hasta el final para cargar contenido dinámico
        scroll_to_bottom(driver)
        time.sleep(1)
        scroll_to_bottom(driver)
        time.sleep(1)

        # Obtener todos los divs con clase 'col-sm-6'
        columns = driver.find_elements(By.CSS_SELECTOR, "div.col-sm-6")

        data = []
        for column in columns:
            # Buscar el enlace dentro del div
            try:
                # Extraer el enlace del artículo dentro del <a> que está después de <span>
                link_element = column.find_element(By.CSS_SELECTOR, "a.uc-card_link")
                link = link_element.get_attribute("href")
                # Si el enlace es relativo, agregar el dominio base
                if link.startswith('/'):
                    link = f"https://www.clapesuc.cl{link}"
            except:
                link = "No disponible"  # Si no hay enlace, asignamos un valor predeterminado

            # Obtener el título
            try:
                title_element = column.find_element(By.CSS_SELECTOR, "h4.h3")
                title = title_element.text.strip()
            except:
                title = "No disponible"

            # Obtener el autor
            try:
                author = column.find_element(By.CSS_SELECTOR, "p.text-weight--bold.no-margin").text.strip()
            except:
                author = "No especificado"

            # Obtener la fecha
            try:
                date = column.find_element(By.CSS_SELECTOR, "p.p-size--sm span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Autor': author,
                'Fecha Publicación': date,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

Seminarios

In [50]:
base_url = "https://clapesuc.cl/seminarios"
seminarios = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=30, headless=False)

Extrayendo artículos de: https://clapesuc.cl/seminarios?page=1
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=2
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=3
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=4
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=5
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=6
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=7
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=8
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=9
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=10
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=11
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=12
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=13
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=14
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=15
Extrayendo artículos de: https://clapesuc.cl/semi

In [53]:
seminarios

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,None,2024-11-29 13:57:31
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,None,2024-11-29 13:57:32
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,None,2024-11-29 13:57:32
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,None,2024-11-29 13:57:32
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,None,2024-11-29 13:57:32
...,...,...,...,...,...,...
254,[Seminario Reforma Tributaria] PPT Principios ...,https://clapesuc.cl/seminarios/seminario-refor...,Claudio Agostini,13 de mayo del 2014,None,2024-11-29 14:00:11
255,[Seminario Educación] PPT Sobre los mitos de ...,https://clapesuc.cl/seminarios/seminario-educa...,Claudia Sanhueza,13 de mayo del 2014,None,2024-11-29 14:00:11
256,[Seminario Educación] PPT Reflexiones sobre l...,https://clapesuc.cl/seminarios/seminario-educa...,Ricardo Paredes,13 de mayo del 2014,None,2024-11-29 14:00:11
257,[Seminario Educación] PPT Gratuidad en Educac...,https://clapesuc.cl/seminarios/seminario-educa...,Sergio Urzúa,13 de mayo del 2014,None,2024-11-29 14:00:11


In [54]:
seminarios.to_excel("seminarios_uc.xlsx", index=False)

Investigación

In [51]:
base_url = "https://clapesuc.cl/investigacion"
investigacion = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=106, headless=False)
investigacion

Extrayendo artículos de: https://clapesuc.cl/investigacion?page=1
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=2
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=3
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=4
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=5
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=6
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=7
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=8
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=9
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=10
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=11
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=12
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=13
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=14
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=15
Extr

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Ideas para el Debate 38: ""Perspectivas para la...",https://clapesuc.cl/investigacion/ideas-para-e...,No especificado,26 de noviembre del 2024,None,2024-11-29 14:00:25
1,Informe: Impacto del Teletrabajo en Productivi...,https://clapesuc.cl/investigacion/informe-impa...,No especificado,18 de noviembre del 2024,None,2024-11-29 14:00:25
2,Informe de empleo formal en el sector privado ...,https://clapesuc.cl/investigacion/informe-de-e...,No especificado,14 de noviembre del 2024,None,2024-11-29 14:00:25
3,"Ideas para el Debate 37: ""Presentaciones del L...",https://clapesuc.cl/investigacion/ideas-para-e...,No especificado,12 de noviembre del 2024,None,2024-11-29 14:00:25
4,Estimación del Valor de la CBA,https://clapesuc.cl/investigacion/estimacion-d...,No especificado,12 de noviembre del 2024,None,2024-11-29 14:00:25
...,...,...,...,...,...,...
943,"Doc.Trabajo Nº2, La utilidad atribuida: una cr...",https://clapesuc.cl/investigacion/doc-trabajo-...,No especificado,3 de junio del 2014,None,2024-11-29 14:09:55
944,No disponible,No disponible,No especificado,No especificada,None,2024-11-29 14:09:55
945,Análisis Impuesto al Capital durante la Gran D...,https://clapesuc.cl/investigacion/analisis-imp...,No especificado,29 de mayo del 2014,None,2024-11-29 14:09:59
946,"Doc.Trabajo Nº1, Reforma Tributaria: Impacto E...",https://clapesuc.cl/investigacion/doc-trabajo-...,No especificado,15 de mayo del 2014,None,2024-11-29 14:10:00


In [55]:
investigacion.to_excel("investigacion_uc.xlsx", index=False)

En los medios

In [62]:
base_url = "https://clapesuc.cl/en-los-medios"
elm = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=125, headless=False)
elm

Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=1
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=2
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=3
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=4
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=5
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=6
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=7
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=8
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=9
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=10
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=11
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=12
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=13
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=14
Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=15
Extr

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Entrevista a Sergio Urzúa, investigador intern...",https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,None,2024-11-29 14:27:37
1,Delincuencia en Chile,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,None,2024-11-29 14:27:37
2,Expertos: no consta que SLEP hayan elevado apr...,https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,None,2024-11-29 14:27:38
3,El legado de Alfonso Swett: un adiós que une a...,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,None,2024-11-29 14:27:38
4,Despidos por necesidad de la empresa anotan nu...,https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,None,2024-11-29 14:27:38
...,...,...,...,...,...,...
1120,Canasta Básica de Alimentos anotaría segunda m...,https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,None,2024-11-29 14:38:12
1121,"Pese a la baja de las tasas largas, los intere...",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,None,2024-11-29 14:38:12
1122,La Región Metropolitana concentra el 25% de la...,https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,None,2024-11-29 14:38:12
1123,El IPC de los adultos mayores supera el nivel ...,https://clapesuc.cl/en-los-medios/el-ipc-de-lo...,Carmen Cifuentes,8 de diciembre del 2023,None,2024-11-29 14:38:12


In [63]:
# Filtrar las filas donde la columna 'Enlace' no sea igual a 'No disponible'
elm = elm[elm['Enlace'] != 'No disponible']
elm

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Entrevista a Sergio Urzúa, investigador intern...",https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,None,2024-11-29 14:27:37
1,Delincuencia en Chile,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,None,2024-11-29 14:27:37
2,Expertos: no consta que SLEP hayan elevado apr...,https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,None,2024-11-29 14:27:38
3,El legado de Alfonso Swett: un adiós que une a...,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,None,2024-11-29 14:27:38
4,Despidos por necesidad de la empresa anotan nu...,https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,None,2024-11-29 14:27:38
...,...,...,...,...,...,...
1119,Ser cesante después de los 50 (y cómo salir de...,https://clapesuc.cl/en-los-medios/ser-cesante-...,Carmen Cifuentes,10 de diciembre del 2023,None,2024-11-29 14:38:12
1120,Canasta Básica de Alimentos anotaría segunda m...,https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,None,2024-11-29 14:38:12
1121,"Pese a la baja de las tasas largas, los intere...",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,None,2024-11-29 14:38:12
1122,La Región Metropolitana concentra el 25% de la...,https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,None,2024-11-29 14:38:12


Indicadores

In [47]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_to_bottom(driver):
    """Desplaza la página hasta el final y espera un segundo para cargar el contenido dinámico."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Esperar un segundo para que la página cargue

def extract_indicators_from_url(driver, url, existing_df=None):
    try:
        driver.get(url)

        # Desplazar hasta el final para cargar contenido dinámico
        scroll_to_bottom(driver)
        time.sleep(1)
        scroll_to_bottom(driver)
        time.sleep(1)

        # Obtener todos los divs con clase 'uc-card card-type--link'
        cards = driver.find_elements(By.CSS_SELECTOR, "div.uc-card.card-type--link")

        data = []
        for card in cards:
            # Extraer el enlace del indicador
            try:
                link_element = card.find_element(By.CSS_SELECTOR, "a.uc-card_link")
                link = link_element.get_attribute("href")
                # Si el enlace es relativo, agregar el dominio base
                if link.startswith('/'):
                    link = f"https://www.clapesuc.cl{link}"
            except:
                link = "No disponible"  # Si no hay enlace, asignamos un valor predeterminado

            # Obtener el título del indicador
            try:
                title_element = card.find_element(By.CSS_SELECTOR, "h4.h3")
                title = title_element.text.strip()
            except:
                title = "No disponible"

            # Obtener la fecha de publicación
            try:
                date = card.find_element(By.CSS_SELECTOR, "p.p-size--sm.p-color--gray.p-text--condensed span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            indicator_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(indicator_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_indicators_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los indicadores, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los indicadores extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_indicators_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo indicadores de: {url}")

        # Llamar a la función extract_indicators_from_url para obtener indicadores de la página
        all_indicators_df = extract_indicators_from_url(driver, url, all_indicators_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_indicators_df is None or len(all_indicators_df) == 0:
            print(f"No se encontraron indicadores en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_indicators_df

In [48]:
base_url = "https://clapesuc.cl/indicadores"
indicadores = extract_indicators_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
indicadores

Extrayendo indicadores de: https://clapesuc.cl/indicadores?page=1


,Título,Enlace,Fecha Publicación,Contenido,Fecha Consulta
0,Índice de Precios de Medicamentos (IPC Med) CL...,https://clapesuc.cl/indicadores/indice-de-prec...,14 de noviembre del 2024,None,2024-11-29 13:56:27
1,Incertidumbre Económica (IEC) y Política Econó...,https://clapesuc.cl/indicadores/indice-de-ince...,14 de noviembre del 2024,None,2024-11-29 13:56:27
2,Probabilidad de Recesión en Chile,https://clapesuc.cl/indicadores/probabilidad-d...,14 de noviembre del 2024,None,2024-11-29 13:56:27
3,Índice de Condiciones del Mercado Laboral (ICML),https://clapesuc.cl/indicadores/indice-de-cond...,14 de noviembre del 2024,None,2024-11-29 13:56:27
4,IPC del Adulto Mayor,https://clapesuc.cl/indicadores/ipc-adulto-mayor,12 de noviembre del 2024,None,2024-11-29 13:56:27
5,Índice de Precios al Consumidor de Escasos Ing...,https://clapesuc.cl/indicadores/indice-de-prec...,8 de noviembre del 2024,None,2024-11-29 13:56:27
6,Precios Inmobiliario Residencial,https://clapesuc.cl/indicadores/indice-inmobil...,9 de octubre del 2024,None,2024-11-29 13:56:27
7,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,21 de septiembre del 2024,None,2024-11-29 13:56:27
8,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,17 de septiembre del 2024,None,2024-11-29 13:56:27
9,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,13 de septiembre del 2024,None,2024-11-29 13:56:27


In [56]:
indicadores.to_excel("indicadores_uc.xlsx", index=False)

In [59]:
import pandas as pd

# Lista de DataFrames
dataframes = [seminarios, indicadores]

# Añadir una columna que indique el origen de cada DataFrame
seminarios['Categoría'] = 'Seminarios'
indicadores['Categoría'] = 'Indicadores'

# Concatenar los DataFrames
df_concatenado0 = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado0

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta,Categoría
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,None,2024-11-29 13:57:31,Seminarios
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...
266,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
267,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
268,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
269,Índice de Acceso a la Vivienda (IAV),https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,None,2024-11-29 13:56:27,Indicadores


In [60]:
df_concatenado0.to_excel("sem_ind_uc.xlsx", index=False)

## Consolidación misma estructura

Agregar investigación al final porque no tiene informacion dentro del link, solo un pdf.

In [ ]:
import pandas as pd

# Lista de DataFrames
dataframes = [seminarios, elm, indicadores]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...
670,Pensiones en el Blanco,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...,"01 de Agosto, 2020",2024-11-29 10:14:34
671,Migración en Chile: Necesidad de una nueva ley,https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...,"28 de Agosto, 2020",2024-11-29 10:14:34
672,Reflexiones sobre el plebiscito,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...,"17 de Septiembre, 2020",2024-11-29 10:14:34
673,Letalidad del COVID-19 y Pobreza Multidimensio...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...,"07 de Octubre, 2020",2024-11-29 10:14:34


## Iteración sobre links

In [89]:
from selenium.webdriver.common.by import By
import time

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (ajustar según la velocidad de carga)
    time.sleep(1)
    
    # Intentar obtener la información:
    
    # Título del seminario (párrafo con texto justificado que contiene un <strong>)
    try:
        title = driver.find_element(By.CSS_SELECTOR, "div p strong").text.strip()
    except:
        title = "No disponible"
    
    # Fecha de publicación: extraída de un contenedor <span>
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "div.row.col-lg-12.align-items-center.mb-36 p span").text.strip()
    except:
        publication_date = "No disponible"
    
    # Contenido del seminario (párrafos con texto justificado)
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "p[align='justify']")
        content = " ".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Filtramos los párrafos vacíos
    except:
        content = "No disponible"

    # Participantes: extraído de los <h4> dentro de <div class="uc-card_body">
    try:
        participants_elements = driver.find_elements(By.CSS_SELECTOR, "div.uc-card_body h4")
        participants = ", ".join([p.text.strip() for p in participants_elements if p.text.strip() != ""])  # Filtramos los elementos vacíos
    except:
        participants = "No disponible"
    
    # Medio de comunicación (Fuente): extraído de un contenedor <p> que contiene "Fuente:"
    try:
        source_text = driver.find_element(By.CSS_SELECTOR, "div.row.col-lg-12.align-items-center.mb-60 p").text.strip()
        # Extraemos la parte después de "Fuente: "
        source = source_text.split("Fuente:")[-1].strip() if "Fuente:" in source_text else "No disponible"
    except:
        source = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Fecha de Publicación": publication_date,
        "Contenido": content,
        "Participantes": participants,
        "Medio de comunicación": source,
        "Enlace": url,
    }
    
    return article_data

In [77]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for _, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            url = row['Enlace']
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


In [90]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y actualiza las filas en las que 'Contenido' esté vacío, llenando las columnas vacías en esas filas.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de los artículos, actualizado.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for index, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            # Verificar si 'Contenido' está vacío
            if pd.isna(row['Contenido']) or row['Contenido'] == "":
                url = row['Enlace']
                # Extraer la información del artículo
                article_data = extract_info_from_url(driver, url)

                if article_data:
                    # Si el contenido se encuentra vacío, actualizar la fila
                    for column, value in article_data.items():
                        if pd.isna(row[column]) or row[column] == "":
                            jg_df.at[index, column] = value

        return jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [93]:
df_combinado_aver = df_combinado.copy()

In [94]:
df_combinado_aver

,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Fecha Consulta,Think Tank
0,09:10,29 de noviembre del 2024,Sergio Urzúa de Clapes UC por Presupuesto 2025...,Sergio Urzúa,Radio Infinita,https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,2024-11-29 14:27:37,CLAPES UC
1,Delincuencia en Chile,27 de noviembre del 2024,Delincuencia en Chile En agosto el Centro Lati...,,Publimetro,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,2024-11-29 14:27:37,CLAPES UC
2,Expertos: no consta que SLEP hayan elevado apr...,27 de noviembre del 2024,Análisis a los Servicios Locales de Educación ...,Carlos Williamson,"El Mercurio - Cuerpo C, Página 5.",https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
3,El legado de Alfonso Swett: un adiós que une a...,26 de noviembre del 2024,El legado de Alfonso Swett: un adiós que une a...,"Felipe Larraín, Alfonso Swett † (1968-2024)",Textual,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
4,Despidos por necesidad de la empresa anotan nu...,26 de noviembre del 2024,Despidos por necesidad de la empresa anotan nu...,Carmen Cifuentes,"La Tercera, Página 3.",https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
...,...,...,...,...,...,...,...,...,...,...
995,Ser cesante después de los 50 (y cómo salir de...,10 de diciembre del 2023,,Carmen Cifuentes,"La Tercera, Página 22",https://clapesuc.cl/en-los-medios/ser-cesante-...,Carmen Cifuentes,10 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
996,Canasta Básica de Alimentos anotaría segunda m...,9 de diciembre del 2023,,,"La Tercera, Página: 3",https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
997,"Pese a la baja de las tasas largas, los intere...",8 de diciembre del 2023,,Hermann González,"El Mercurio - Cuerpo B, Página: 4",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC


In [95]:
aver = df_combinado_aver[998:]
aver

,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Fecha Consulta,Think Tank
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
999,El IPC de los adultos mayores supera el nivel ...,8 de diciembre del 2023,,Carmen Cifuentes,"El Mercurio - Cuerpo B, Página: 2",https://clapesuc.cl/en-los-medios/el-ipc-de-lo...,Carmen Cifuentes,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC


In [96]:
averinfo = process_all_links(aver)
averinfo

Extrayendo datos de artículos: 100%|██████████| 2/2 [00:06<00:00,  3.03s/it]


,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Fecha Consulta,Think Tank
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,Revisión de estudios realizados por investigad...,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
999,El IPC de los adultos mayores supera el nivel ...,8 de diciembre del 2023,De acuerdo con estudio elaborado por : El IPC ...,Carmen Cifuentes,"El Mercurio - Cuerpo B, Página: 2",https://clapesuc.cl/en-los-medios/el-ipc-de-lo...,Carmen Cifuentes,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC


Para todos pero dara error para varias filas por fecha y programa, correr de nuevo con esa estructura de error.

In [79]:
df_info = process_all_links(elm)
df_info

Extrayendo datos de artículos: 100%|██████████| 1000/1000 [35:48<00:00,  2.15s/it]


,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace
0,09:10,29 de noviembre del 2024,Sergio Urzúa de Clapes UC por Presupuesto 2025...,Sergio Urzúa,Radio Infinita,https://clapesuc.cl/en-los-medios/entrevista-a...
1,Delincuencia en Chile,27 de noviembre del 2024,Delincuencia en Chile En agosto el Centro Lati...,,Publimetro,https://clapesuc.cl/en-los-medios/delincuencia...
2,Expertos: no consta que SLEP hayan elevado apr...,27 de noviembre del 2024,Análisis a los Servicios Locales de Educación ...,Carlos Williamson,"El Mercurio - Cuerpo C, Página 5.",https://clapesuc.cl/en-los-medios/expertos-no-...
3,El legado de Alfonso Swett: un adiós que une a...,26 de noviembre del 2024,El legado de Alfonso Swett: un adiós que une a...,"Felipe Larraín, Alfonso Swett † (1968-2024)",Textual,https://clapesuc.cl/en-los-medios/el-legado-de...
4,Despidos por necesidad de la empresa anotan nu...,26 de noviembre del 2024,Despidos por necesidad de la empresa anotan nu...,Carmen Cifuentes,"La Tercera, Página 3.",https://clapesuc.cl/en-los-medios/despidos-por...
...,...,...,...,...,...,...
995,Ser cesante después de los 50 (y cómo salir de...,10 de diciembre del 2023,,Carmen Cifuentes,"La Tercera, Página 22",https://clapesuc.cl/en-los-medios/ser-cesante-...
996,Canasta Básica de Alimentos anotaría segunda m...,9 de diciembre del 2023,,,"La Tercera, Página: 3",https://clapesuc.cl/en-los-medios/canasta-basi...
997,"Pese a la baja de las tasas largas, los intere...",8 de diciembre del 2023,,Hermann González,"El Mercurio - Cuerpo B, Página: 4",https://clapesuc.cl/en-los-medios/pese-a-la-ba...
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...


In [80]:
df_concatenado = elm.copy()

In [81]:
# Identificar las columnas que están en df_concatenado pero no en df_info
columnas_extra = [col for col in df_concatenado.columns if col not in df_info.columns]

# Seleccionar solo las columnas extra de df_concatenado
df_concatenado_extra = df_concatenado[['Enlace'] + columnas_extra]

# Hacer el merge entre df_info y df_concatenado_extra en la columna "Enlace"
df_combinado = pd.merge(df_info, df_concatenado_extra, on='Enlace', how='left')

# Ver el DataFrame combinado
df_combinado

,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Fecha Consulta
0,09:10,29 de noviembre del 2024,Sergio Urzúa de Clapes UC por Presupuesto 2025...,Sergio Urzúa,Radio Infinita,https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,2024-11-29 14:27:37
1,Delincuencia en Chile,27 de noviembre del 2024,Delincuencia en Chile En agosto el Centro Lati...,,Publimetro,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,2024-11-29 14:27:37
2,Expertos: no consta que SLEP hayan elevado apr...,27 de noviembre del 2024,Análisis a los Servicios Locales de Educación ...,Carlos Williamson,"El Mercurio - Cuerpo C, Página 5.",https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,2024-11-29 14:27:38
3,El legado de Alfonso Swett: un adiós que une a...,26 de noviembre del 2024,El legado de Alfonso Swett: un adiós que une a...,"Felipe Larraín, Alfonso Swett † (1968-2024)",Textual,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,2024-11-29 14:27:38
4,Despidos por necesidad de la empresa anotan nu...,26 de noviembre del 2024,Despidos por necesidad de la empresa anotan nu...,Carmen Cifuentes,"La Tercera, Página 3.",https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,2024-11-29 14:27:38
...,...,...,...,...,...,...,...,...,...
995,Ser cesante después de los 50 (y cómo salir de...,10 de diciembre del 2023,,Carmen Cifuentes,"La Tercera, Página 22",https://clapesuc.cl/en-los-medios/ser-cesante-...,Carmen Cifuentes,10 de diciembre del 2023,2024-11-29 14:38:12
996,Canasta Básica de Alimentos anotaría segunda m...,9 de diciembre del 2023,,,"La Tercera, Página: 3",https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,2024-11-29 14:38:12
997,"Pese a la baja de las tasas largas, los intere...",8 de diciembre del 2023,,Hermann González,"El Mercurio - Cuerpo B, Página: 4",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,2024-11-29 14:38:12
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,2024-11-29 14:38:12


In [82]:
df_combinado["Think Tank"] = "CLAPES UC"

In [97]:
df_combinado_info = process_all_links(df_combinado_aver)
df_combinado_info

Extrayendo datos de artículos: 100%|██████████| 1000/1000 [24:42<00:00,  1.48s/it]


,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Fecha Consulta,Think Tank
0,09:10,29 de noviembre del 2024,Sergio Urzúa de Clapes UC por Presupuesto 2025...,Sergio Urzúa,Radio Infinita,https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,2024-11-29 14:27:37,CLAPES UC
1,Delincuencia en Chile,27 de noviembre del 2024,Delincuencia en Chile En agosto el Centro Lati...,,Publimetro,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,2024-11-29 14:27:37,CLAPES UC
2,Expertos: no consta que SLEP hayan elevado apr...,27 de noviembre del 2024,Análisis a los Servicios Locales de Educación ...,Carlos Williamson,"El Mercurio - Cuerpo C, Página 5.",https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
3,El legado de Alfonso Swett: un adiós que une a...,26 de noviembre del 2024,El legado de Alfonso Swett: un adiós que une a...,"Felipe Larraín, Alfonso Swett † (1968-2024)",Textual,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
4,Despidos por necesidad de la empresa anotan nu...,26 de noviembre del 2024,Despidos por necesidad de la empresa anotan nu...,Carmen Cifuentes,"La Tercera, Página 3.",https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,2024-11-29 14:27:38,CLAPES UC
...,...,...,...,...,...,...,...,...,...,...
995,Ser cesante después de los 50 (y cómo salir de...,10 de diciembre del 2023,Ser cesante después de los 50 (y cómo salir de...,Carmen Cifuentes,"La Tercera, Página 22",https://clapesuc.cl/en-los-medios/ser-cesante-...,Carmen Cifuentes,10 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
996,Canasta Básica de Alimentos anotaría segunda m...,9 de diciembre del 2023,Canasta Básica de Alimentos anotaría segunda m...,,"La Tercera, Página: 3",https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
997,"Pese a la baja de las tasas largas, los intere...",8 de diciembre del 2023,"No obstante, algunos bancos han activado ofert...",Hermann González,"El Mercurio - Cuerpo B, Página: 4",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,Revisión de estudios realizados por investigad...,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,2024-11-29 14:38:12,CLAPES UC


In [98]:
df_combinado_info.columns

Index(['Título', 'Fecha de Publicación', 'Contenido', 'Participantes',
       'Medio de comunicación', 'Enlace', 'Autor', 'Fecha Publicación',
       'Fecha Consulta', 'Think Tank'],
      dtype='object')

In [99]:
df_combinado_info = df_combinado_info[['Título', 'Fecha de Publicación', 'Contenido', 'Participantes',
       'Medio de comunicación', 'Enlace', 'Autor', 'Fecha Publicación', 'Think Tank']]
df_combinado_info

,Título,Fecha de Publicación,Contenido,Participantes,Medio de comunicación,Enlace,Autor,Fecha Publicación,Think Tank
0,09:10,29 de noviembre del 2024,Sergio Urzúa de Clapes UC por Presupuesto 2025...,Sergio Urzúa,Radio Infinita,https://clapesuc.cl/en-los-medios/entrevista-a...,Sergio Urzúa,29 de noviembre del 2024,CLAPES UC
1,Delincuencia en Chile,27 de noviembre del 2024,Delincuencia en Chile En agosto el Centro Lati...,,Publimetro,https://clapesuc.cl/en-los-medios/delincuencia...,Clapes UC,27 de noviembre del 2024,CLAPES UC
2,Expertos: no consta que SLEP hayan elevado apr...,27 de noviembre del 2024,Análisis a los Servicios Locales de Educación ...,Carlos Williamson,"El Mercurio - Cuerpo C, Página 5.",https://clapesuc.cl/en-los-medios/expertos-no-...,Carlos Williamson,27 de noviembre del 2024,CLAPES UC
3,El legado de Alfonso Swett: un adiós que une a...,26 de noviembre del 2024,El legado de Alfonso Swett: un adiós que une a...,"Felipe Larraín, Alfonso Swett † (1968-2024)",Textual,https://clapesuc.cl/en-los-medios/el-legado-de...,Alfonso Swett,26 de noviembre del 2024,CLAPES UC
4,Despidos por necesidad de la empresa anotan nu...,26 de noviembre del 2024,Despidos por necesidad de la empresa anotan nu...,Carmen Cifuentes,"La Tercera, Página 3.",https://clapesuc.cl/en-los-medios/despidos-por...,Carmen Cifuentes,26 de noviembre del 2024,CLAPES UC
...,...,...,...,...,...,...,...,...,...
995,Ser cesante después de los 50 (y cómo salir de...,10 de diciembre del 2023,Ser cesante después de los 50 (y cómo salir de...,Carmen Cifuentes,"La Tercera, Página 22",https://clapesuc.cl/en-los-medios/ser-cesante-...,Carmen Cifuentes,10 de diciembre del 2023,CLAPES UC
996,Canasta Básica de Alimentos anotaría segunda m...,9 de diciembre del 2023,Canasta Básica de Alimentos anotaría segunda m...,,"La Tercera, Página: 3",https://clapesuc.cl/en-los-medios/canasta-basi...,Clapes UC,9 de diciembre del 2023,CLAPES UC
997,"Pese a la baja de las tasas largas, los intere...",8 de diciembre del 2023,"No obstante, algunos bancos han activado ofert...",Hermann González,"El Mercurio - Cuerpo B, Página: 4",https://clapesuc.cl/en-los-medios/pese-a-la-ba...,Hermann González,8 de diciembre del 2023,CLAPES UC
998,La Región Metropolitana concentra el 25% de la...,8 de diciembre del 2023,Revisión de estudios realizados por investigad...,Hernán de Solminihac,"El Mercurio - Cuerpo A, Página: 10",https://clapesuc.cl/en-los-medios/la-region-me...,Hernán de Solminihac,8 de diciembre del 2023,CLAPES UC


In [100]:
df_combinado_info.to_excel("CLAPES UC - 2.xlsx", index = False)